# TESLA Model S Induction Motor Design

## 1. Introduction

Model S is introduced in June 2012 by Tesla Motors. It has been sold over 100k by December 2015. The very first model with 85 kWh battery pack has a range of 426 km.

Induction motor used in this model is required to be designed using analytical calculations and motoranalysis tool in Matlab.

See [Tire Dimensions](https://www.mcgrefer.com/sizeinfo/2454519)

See [Model S Specs](https://en.wikipedia.org/wiki/Tesla_Model_S)

See [Presentation](http://sites.ieee.org/miami/files/2014/11/Hendershot-FIU-Lecture.pdf)

## 2. Design Specifications and Basic Calculations

In [1]:
import math
gr=9.73 #Gear ratio of Tesla
wr=0.35155 #Wheel radius of Tesla
Max_spd=230 #Max speed in km/h
Max_speed=Max_spd*1000/3600 #Max speed in m/s
Ang_speed=Max_speed*2*math.pi*wr #Max angular velocity
Motor_speed=Ang_speed*gr #Motor speed in rad/sed
n=Motor_speed*60/(2*math.pi) #Max speed of motor in rpm
print ('Max motor speed is',round(n),'rpm')

Max motor speed is 13112 rpm


In [2]:
Bat_volt=400 #Battery voltage in Volts
Vfund=(Bat_volt/2)*(4/math.pi) #Fundamental phase voltage
Vrms=Vfund*math.sqrt(3)/math.sqrt(2) #Rms line voltage
print ('Line voltage is', round(Vrms),'V')

Line voltage is 312 V


For vector controlled inverters the line voltage is calculated as follows:

In [3]:
Vphase_peak=Bat_volt/math.sqrt(3)
Vline=Vphase_peak*math.sqrt(3)/math.sqrt(2) #Rms line voltage for vector controlled drive
Vphase=Vphase_peak/math.sqrt(2) #Rms phase voltage
print ('Line voltage is', round(Vline),'V')

Line voltage is 283 V


Initially, let the power factor and efficiency be as follows:

In [4]:
eff=0.95 #Rated efficiency
pf=0.85 #Power factor

In [5]:
Pmax=285000 #Max power of motor in Watts
Imax=Pmax/(math.sqrt(3)*Vline*eff*pf)
print ('Max motor current is', round(Imax),'A')

Max motor current is 720 A


Pole number is selected as 4 and the machine is known to be a 3 phase machine

In [6]:
pp=2 #Pole pair
p=4 #Pole number
m=3 #Phase number

In [7]:
fmax=n*p/120 #Max frequency of motor
print ('Maximum operating frequency is', round(fmax), 'Hz')

Maximum operating frequency is 437 Hz


For a more realistic operating frequency (rated frequency), use 90 km/h as rated speed and recalculate the stator frequency

In [8]:
Rtd_speed=90 #Rated speed in km/h
f=fmax/Max_spd*Rtd_speed #Nominal frequency
print ('Nominal frequency is', round(f), 'Hz')

Nominal frequency is 171 Hz


## 3. Main Dimensions of Stator Core

In [9]:
KE=0.98-0.005*pp
Sgap=KE*Pmax/(eff*pf)
print ('KE=',KE)
print ('Sgap=',round(Sgap,2))

KE= 0.97
Sgap= 342352.94


In [10]:
C0=240e3 # Esson's constant in J/m^3
lambda1=1.5 # Stack aspect ratio
Dis=math.pow((2.0*pp*pp*Sgap)/(math.pi*lambda1*f*C0),1.0/3.0) # Stator bore diameter in m
print ('Stator bore diameter is', round(Dis,4)*100, 'cm')

Stator bore diameter is 24.19 cm


In [11]:
tau=math.pi*Dis/(2.0*pp) # Pole pitch in m
L=lambda1*tau # Stack length in m
print ('tau=', round(tau,3))
print ('L=', round(L,2))

tau= 0.19
L= 0.29


For a better performance, stator slots per pole per phase q is selected as 4. 

Slot pitch tau_s is calculated as follows:

In [12]:
q=4.0
tau_s=tau/(m*q) # Slot pitch in m
print ('tau_s=', round(tau_s,4))

tau_s= 0.0158


For 2pp=4, KD is chosen as 0.62.

In [13]:
KD=0.62 # Dis/Dout
Dout=Dis/KD # Stator outer diameter in m
print ('Dout=', round(Dout,2))

Dout= 0.39


In [14]:
g=(0.1+0.012*math.pow(Pmax,1.0/3.0))*1e-3 # Air gap length in m for 2pp>2
print ('g=', round(g,4)*1000, 'mm')

g= 0.9 mm


Too small airgap would produce large space airgap field harmonics and additional losses while a too large one would reduce the power factor and efficiency.

The airgap found above is a moderate one to be used in later steps.

## 4. The Stator Winding

Two-layer winding with chorded coils, y/tau_s=10/12 is chosen 10/12=0.83 to reduce the 5th harmonic in mmf.

In [15]:
Ns=p*q*m # Number of stator slots
print (Ns)

48.0


In [16]:
alpha_ec=2.0*math.pi*pp/Ns # Electrical angle between emfs in neighboring slots in rad
print (round(alpha_ec,2))

0.26


In [17]:
Kq1=math.sin(q*alpha_ec/2.0)/(q*math.sin(alpha_ec/2.0)) # Distribution factor
Ky1=math.sin(math.pi/2.0*10.0/12.0) # Pitch factor
Kw1=Kq1*Ky1 # Winding factor
print (round(Kq1,2))
print (round(Ky1,2))
print (round(Kw1,2))

0.96
0.97
0.93


In [18]:
Bg=0.65 #Air gap flux density in T
Kf=1.085
alpha_i=0.729 # Pole spanning coefficient
phi=alpha_i*tau*L*Bg # Pole flux in Wb
print (round(phi,4))

0.0257


In [19]:
W1=KE*Vphase/(4.0*Kf*Kw1*f*phi) # Number of turns per phase
print (round(W1,2))

8.99


In [20]:
a1=1.0 # Number of current paths in parallel
ns=a1*W1/(pp*q) # Number of conductors per slot
print (ns)

1.1237695356670816


ns should be an even number as there are two distinct coils per slot in a
double layer winding. Choose ns=2, then recalculate W1 and Bg

In [21]:
ns=2
W1_new=pp*q*ns
Bg=Bg*W1/W1_new
print (W1_new)
print (Bg)

16.0
0.3652250990918015


Such a low Bg value is not applicable. Increase Bg by changing a1 value.

In [22]:
Iln=Imax #Rated current
Jcos=6.0 # Current density in A/mm^2
Aco=Iln/(Jcos*a1) # Magnetic wire cross section in mm^2
rco=math.sqrt(Aco/math.pi) #Wire radius in mm
dco=2*rco
print (round(Aco,2))
print (round(rco,2))
print (round(dco,2))

120.07
6.18
12.36


Although frequency is not high, check that this value is similar to the skin depth at the deisgn frequency.

Since dco>1.3 mm parallel conductors should be used 

Use 120 conductors in parallel, ap=120

In [23]:
ap=120
dco=math.sqrt(4.0*Aco/(math.pi*ap))
print (round(dco,2))

1.13


Since ap>4, a1 should be modified (increased) as follows.

Each coil is made of ns/ap turns.

Wire diameter shall be selected here.

## 5. Stator Slot Sizing

In [24]:
Kfill=0.44 # Fill factor for round wire
Asu=math.pi*dco*dco*ap*ns/(4.0*Kfill) # Useful slot area in mm^2
print (round(Asu,2))

545.79
